# Molecular Dynamics Demonstration with ASE

Here we will go through an example of a MD simulation by using the Atomic Simulation Environment (ASE). Most of this tutorial was taken from the [ASE Workshop 2023](https://ase-workshop-2023.github.io/tutorial/) which nicely covers this topic.

## ASE Basics

ASE is similar to Pymatgen, as it also conveniently generates molecules and crystal structures and is capable of manipulating them and reading/saving them from common crystal/molecule files.

In [ ]:
from ase import Atoms
from ase.visualize import view

a = 5.387
crystal = Atoms('Zn4S4',
                scaled_positions=[[0., 0., 0.],
                           [0., 0.5, 0.5],
                           [0.5, 0., 0.5],
                           [0.5, 0.5, 0.],
                           [0.25, 0.75, 0.75],
                           [0.25, 0.25, 0.25],
                           [0.75, 0.75, 0.25],
                           [0.75, 0.25, 0.75]],
               cell=[a, a, a],
               pbc=True)

view(crystal, viewer='ngl')

In [ ]:
print(crystal.get_positions(), end='\n\n')  # cartesian coordinates
print(crystal.get_scaled_positions(), end='\n\n')  # fractional coordinates
print(crystal.get_chemical_symbols())
print(crystal.cell.cellpar())
print(crystal.cell.array)  # cell vectors

Similar to Pymatgen, `Atoms` objects are lists of `Atom`s (in Pymatgen, a `Structure` is a list of `PeriodicSite`s).

In [ ]:
for atom in crystal:
    print(atom.symbol, atom.position, atom.mass)

In [ ]:
zinc_indices = [i for i, atom in enumerate(crystal) if atom.symbol == 'Zn']
zinc_sublattice = crystal[zinc_indices]
view(zinc_sublattice, viewer='ngl')

In [ ]:
from ase import Atom

defected_structure = crystal.copy()
del defected_structure[0]
defected_structure.append(Atom('O', position=(1, 3, 4.04025)))
view(defected_structure, viewer='ngl')

ASE has many convenience function to build known structures.

For example, for the `bulk`, it can easily create the following structures: sc, fcc, bcc, tetragonal, bct, hcp, rhombohedral, orthorhombic, mcl, diamond, zincblende, rocksalt, cesiumchloride, fluorite or wurtzite.

In [ ]:
from ase.build import bulk, nanotube, graphene_nanoribbon, fcc111

a1 = bulk('Cu', 'fcc', a=3.6)
a2 = bulk('Cu', 'fcc', a=3.6, cubic=True)
# view(a1, viewer='ngl')
# view(a2, viewer='ngl')

diamond = bulk('C', 'diamond', a=3.57)
# view(diamond, viewer='ngl')

cnt = nanotube(6, 6, length=10)
# view(cnt, viewer='ngl')

gnr = graphene_nanoribbon(6, 10, type='zigzag', saturated=True)
# view(gnr, viewer='ngl')

slab = fcc111('Pt', size=(2, 2, 4), vacuum=10.0)
view(slab, viewer='ngl')


## MD Demonstration Using ASE's `Calculator` Class

A `Calculator` object can be attached to an `Atoms` object.

The `Calculator` takes the atomic numbers and positions from `Atoms` and calculates basic properties such as energy or forces.

In [ ]:
import ase.build
from ase.calculators.emt import EMT
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

# Set up a crystal
cu_cube = ase.build.bulk('Cu', cubic=True) * [3, 3, 3]

# Describe the interatomic interactions with the Effective Medium Theory
cu_cube.calc = EMT()

# Set the momenta corresponding to T=300K
MaxwellBoltzmannDistribution(cu_cube, temperature_K=300)

In [ ]:
row_limit = 4
print('velocities:')
print(cu_cube.get_velocities()[:row_limit])

print(f'\nkinetic energy: {cu_cube.get_kinetic_energy()}')

print('\nforces:')
print(cu_cube.get_forces()[:row_limit])

In [ ]:
from ase import units
from ase.md.verlet import VelocityVerlet

dyn = VelocityVerlet(cu_cube, 5 * units.fs)

In [ ]:
dyn.run(1)

print('velocities:')
print(cu_cube.get_velocities()[:row_limit])

print('\nforces:')
print(cu_cube.get_forces()[:row_limit])

In [ ]:
from ase import Atoms

def printenergy(atoms: Atoms) -> None:
    """Function to print the potential, kinetic and total energy"""
    epot = atoms.get_potential_energy() / len(atoms)
    ekin = atoms.get_kinetic_energy() / len(atoms)
    temperature = ekin / (1.5 * units.kB)

    print(f'Energy per atom: Epot = {epot:.3f}eV  Ekin = {ekin:.3f}eV '
          f'(T={temperature:3.0f}K)  Etot = {epot+ekin:.3f}eV')

# print starting energies
printenergy(cu_cube)
# print energies as system evolves
for i in range(20):
    dyn.run(10)
    printenergy(cu_cube)

In [ ]:
from functools import partial
dyn.attach(partial(printenergy, cu_cube), interval=10)
dyn.run(100)

In [ ]:
import numpy as np
ref_atoms = ase.build.bulk('Cu', cubic=True) * [3, 3, 3]
ref_atoms.center()  # Shift centre-of-mass to the origin

def rms_observer():
    cu_cube.center()  # Remove drift, make consistent with ref
    displacements = cu_cube.positions - ref_atoms.positions
    rms_displacements = np.sqrt((displacements**2).mean(axis=0))
    print(rms_displacements)

dyn = VelocityVerlet(cu_cube, 5 * units.fs)
dyn.attach(rms_observer, interval=50)
dyn.run(400)

In [ ]:
from ase.visualize import view

view(cu_cube, viewer='ngl')

In [ ]:
from ase.io.trajectory import Trajectory
from ase.md import Langevin

cu_lump = cu_cube.copy()
cu_lump.pbc=False
cu_lump.calc = EMT()

def energy_observer():
    printenergy(cu_lump)

dyn = Langevin(cu_lump, 5 * units.fs, friction=0.005, temperature_K=1000)
dyn.attach(energy_observer, interval=50)

# We also want to save the positions of all atoms after every 50th time step.
traj = Trajectory('cu_melt.traj', 'w', cu_lump)
dyn.attach(traj.write, interval=50)  # type: ignore

# Now run the dynamics
dyn.run(1000)

In [ ]:
import ase.io

frames = ase.io.read('cu_melt.traj', index=':')
view(frames, viewer='ngl')

In [ ]:
for temperature in range(800, 100, -50):
    dyn.set_temperature(temperature_K=temperature)
    dyn.run(100)

frames = ase.io.read('cu_melt.traj', index=':')
view(frames, viewer='ngl')